In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Поведенческие паттерны  
  
  
### Определяют алгоритмы и способы реализации взаимодействия различных объектов и классов.

* Chain of Responsibility
* Command
* Iterator
* Mediator
* __Memento__
* Observer
* __State__
* Strategy
* Template Method
* __Visitor__

---

## Memento

### Задачи
  
  1. Вы создаёте IDE с возможностью сохранять этапы проекта (например, в виде diff-а) и, если нужно, откатываться до них (некая система контроля версий на минималках).
  2. При обновлении своей ОС (или при переносе ее на другое устройство) вы делаете backup.
  3. В компьютерных играх необходимо время от времени сохранять ваш прогресс - так называемые "чекпоинты".
  
Во всех случаях задача стоит следующая: "сохранить и/или восстановить прошлые состояния объектов, не раскрывая их подробности реализации, т.е. не нарушая инкапсуляцию".

### Решение  
  
Давайте поручим создание копии состояния объекта `Originator` самому этому объекту, который этим состоянием владеет. Вместо того, чтобы делать снимок «извне», наш объект сам сделает копию своих полей, ведь ему доступны все поля, даже приватные. **Kакой порождающий паттерн это Вам напоминает?**  
  
Дополнительно давайте создадим специальное хранилище (`Caretaker`), которое как раз и будет держать копию состояния в специальном объекте-снимке. С другой стороны, снимок (`Memento`) должен быть открыт для своего создателя `Originator`, позволяя прочесть и восстановить его внутреннее состояние, т.е. клиентский код посредством обращения к `Caretaker` может откатить состояние `Originator` на какое-то из прошлых (тут уже вступает в силу Ваша бизнесс-логика).

![memento](uml/memento.png)

### Плюсы

 * не нарушает инкапсуляции
 * объекту не надо самому хранить историю версий

### Минусы

 * на хранение истории списков может потребоваться много памяти
 * надо не забывать чистить слишком старые снимки

---

## State

### Задачa

Механика получения сообщений в приложении соцсети меняется в зависимости от того, в каком состоянии оно находится в телефоне.
  * Если приложение закрыто — приходит пуш-уведомление
  * Если открыто — новое сообщение появляется в блоке сообщений
  * Если свёрнуто — телефон просто издаёт звуковой сигнал
  
Случаи применения:
  * Объект, поведение которого кардинально меняется от состояния
  * Много похожих if-ов, где выбирается поведение
  
**Kакой поведенческий паттерн это Вам напоминает? Почему он не решает эту задачу?**

### Решение  
  
Паттерн Состояние предлагает создать отдельные классы для каждого состояния, в котором может пребывать объект, а затем вынести туда поведения, соответствующие этим состояниям.  
  
Вместо того, чтобы хранить код всех состояний, первоначальный объект, называемый контекстом, будет содержать ссылку на один из объектов-состояний и делегировать ему работу, зависящую от состояния.

![state](uml/state.png)
  
Очень важным нюансом, отличающим этот паттерн от Стратегии, является то, что и контекст, и сами конкретные состояния могут знать друг о друге и инициировать переходы от одного состояния к другому (объект может переключать свое состояние, а в Strategy такой возможности просто нет!).

### Плюсы

  * Избегаем много if-ов
  * Концентрация кода по одному состоянию
  * Упрощение кода в целом
 
### Минусы

 * Не надо делать, когда условия совсем простые

---

## Visitor

### Задача

Вы реализуете приложение для доставки еды и продуктов.  
В нём есть несколько типов пользователей — администратор, курьер и клиент и несколько типов заведений — склад, ресторан, магазин. При этом поведение каждого пользователя в зависимости от заведения различно и наоборот.

### Решение  

Паттерн Посетитель предлагает разместить новое поведение в отдельном классе, вместо того чтобы множить его сразу в нескольких классах. Объекты, с которыми должно было быть связано поведение, не будут выполнять его самостоятельно. Вместо этого вы будете передавать эти объекты в методы посетителя.  
  
Код поведения, скорее всего, должен отличаться для объектов разных классов, поэтому и методов у посетителя должно быть несколько. Названия и принцип действия этих методов будет схож, но основное отличие будет в типе принимаемого в параметрах объекта, например:

```c++
class AdminVisitor {
  public:
    void VisitWarehouse(Warehouse& warehouse);
    void VisitRestaurant(Restaurant& restaraunt);
    void VisitShop(Shop& shop);
    ...
}
```
  
Здесь возникает вопрос: как подавать узлы в объект-посетитель? Так как все методы имеют отличающуюся сигнатуру, использовать полиморфизм при выборе помещения не получится. Придётся проверять тип узлов для того, чтобы выбрать соответствующий метод посетителя:

```c++
void Previsit(Visitor* visitor, Placement* placement) {
    if (reflection::IsInstanceOf(*placement, Warehouse)) {
        visitor.VisitWarehouse(*placement);
    } else if (reflection::IsInstanceOf(*placement, Restaurant)) {
        visitor.VisitRestaurant(*placement);
    } else if (reflection::IsInstanceOf(*placement, Shop)) {
        visitor.VisitShop(*placement);
    } else {
        throw new LogicalException(...);
    }
}
```

Но паттерн Посетитель решает и эту проблему, используя механизм _двойной диспетчеризации_. Вместо того, чтобы самим искать нужный метод, мы можем поручить это объектам, которые передаём в параметрах посетителю. А они уже вызовут правильный метод посетителя.

```c++
class AdminVisitor {
  public:
    void Visit(Warehouse& warehouse);
    void Visit(Restaurant& restaraunt);
    void Visit(Shop& shop);
    ...
}

class Placement {
  public:
    void Accept(Visitor& visitor) {
        visitor.Visit(this);   
    }
    ...
}

class Warehouse: public Placement { ... };
    
class Restaurant: public Placement { ... };
    
class Shop: public Placement { ... };

...

void SomeBusinessLogin() {
    ...
    Placement* placent = GetPlacementFromSomewhere(...);
    AdminVisitor adminVisitor;
    placent->Accept(adminVisitor);
    ...
}
```

![visitor](uml/visitor.png)

### Плюсы

 * Издатели не зависят от конкретных классов подписчиков и наоборот
 * Подхватывать и отписывать объекты на лету

---